## Import libraries

In [1]:
from pathlib import Path
import pandas as pd
import openpyxl
import xlwings as xw

In [68]:
#Access input folder
input_dir1 = Path ("G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_final")
#print ("INPUT 1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder1 for folder1 in input_dir1.iterdir() if folder1.is_dir()]
#print ("INPUT 2",input_dir2)

# Create output directory
output_dir1 = Path("G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets")
output_dir1.mkdir(exist_ok = True)

# Define current work directory
current_dir = Path.cwd()

# Access the template to be used to generate files
excel_template = current_dir/ "Template.xlsx"

# Mapping dictionary
mapping = {"1": 0, "2": 15, "3": 30, "4": 45, "5": 60}

## Open RP_all_all excel file

In [69]:
# Load the reference DataFrame for change time and trough time
reference_df = pd.read_excel('RP_all_all_dummy.xlsx')

# Define the function to create the phase array
def get_phase_array(time_array, pressure_array, SD, EM, A):
    # Find the row in the reference dataframe matching SD, EM, and A
    row = reference_df[(reference_df['Standoff distance'] == SD) & (reference_df['Mass'] == EM) & (reference_df['Angle'] == A)]
    
    if row.empty:
        raise ValueError(f"No matching entry found for SD={SD}, EM={EM}, A={A}")

    change_time = round(row['Change time'].values[0],10)
    change_pressure = row['CP'].values[0]
    trough_time = round(row['Trough time'].values[0],10)
    trough_pressure = row['TP'].values[0]
    a=0
    b=0
    phase_array = []
    for t, p in zip(time_array, pressure_array):
        if round (t, 10) < change_time:
            phase_array.append('positive')#{t} {change_time}')
        elif change_time <= round (t, 10) < trough_time:
            a+=1
            if a == 1 and abs(p - change_pressure) <0.000001:
                phase_array.append('negative_increasing')
            elif a == 1 and abs(p - change_pressure) >0.000001:
                phase_array.append('error')
                print (f'error- p={p} , change p ={change_pressure}, t ={t}, change t = {change_time}')
            else:
                phase_array.append('negative_increasing')# {t} {trough_time}')
    
        elif trough_time <= round (t, 10):
            b+=1
            if b == 1 and abs(p - trough_pressure) < 0.000001:
                phase_array.append('negative_decreasing')
            elif b == 1 and abs(p - trough_pressure) > 0.000001:
                phase_array.append('error')
                print (f'error- p={p} , trough p ={trough_pressure}, t ={t}, trough t = {trough_time}')
            else:
                phase_array.append('negative_decreasing')
        else:
            phase_array.append('something wrong')
            print ('something wrong')
            
    
    return phase_array

In [70]:
for folder1 in input_dir2: # folder1-RP_curves_fr5m

    input_dir3 =  [folder2 for folder2 in folder1.iterdir() if folder2.is_dir()]
    # Create output folders
    output_dir2 = output_dir1/ folder1.name
    output_dir2.mkdir(exist_ok = True)
    #print ("OUTPUT 2", output_dir2)
    
    for folder2 in input_dir3: # folder2-05m00.5kg
        
        SD = int(folder2.name[0:2])
        EM = float(folder2.name[3:-2])
        
        # Create output folders
        output_dir3 = output_dir2/ folder2.name
        output_dir3.mkdir(exist_ok = True)
        #print ("OUTPUT 3", output_dir3)
        
        # Make a list of data file names
        files1 = [file1 for file1 in folder2.rglob("*") if file1.suffix == '.xlsx' and file1.suffix != '.png']
    
        # Initiate xlwings library
        with xw.App (visible = False) as app:
            
            # Access each file 
            for file1 in files1: # file-1.xlsx
                
                # Open excel template
                wb = app.books.open(excel_template)
                
                # Read data from Excel file
                df1 = pd.read_excel(file1)
                
                # Extract time and pressure columns
                time = df1.iloc[:, 3].dropna().values
                pressure = df1.iloc[:, 4].dropna().values
                
                # Assign value to A based on file.stem
                file_name = file1.stem
                A = mapping.get(file_name, None)
                #print(file_name, A)
                
                # Generate the phase array
                try:
                    phase = get_phase_array(time, pressure, SD, EM, A)
                except ValueError as e:
                    print(e)
                    continue  # Skip this file if no matching entry is found
                    
                # Check for "error" in phase array
                if 'error' in phase:
                    print(f"Error in this file - SD={SD}, EM={EM}, A={A}")
                
                # Create the DataFrame with the specified columns
                df2= pd.DataFrame({
                    "Mass": [EM] * len(time),
                    "Standoff distance": [SD] * len(time),
                    "Angle": [A] * len(time),
                    "Time": time,
                    "Phase": phase,
                    "Pressure": pressure
                })
                
                # Save df2 to an Excel file in output_dir3 with the same name as file1
                output_file_path = output_dir3 / file1.name
                df2.to_excel(output_file_path, index=False)
                #print(f"Saved processed data to {output_file_path}")

error- p=0.005492190000005059 , change p =-0.0005234400000023176, t =83.719544247, change t = 83.719544247
Error in this file - SD=29, EM=0.5, A=0
error- p=0.009476559999988587 , change p =-0.00512499999999988, t =81.226328522, change t = 81.226328522
Error in this file - SD=29, EM=4.5, A=0
error- p=2.546709896888895 , change p =-0.360843749999994, t =14.7453091294, change t = 14.7453091294
error- p=2.4107583348888966 , trough p =-5.648210937999998, t =14.9983811378, trough t = 14.9983811378
Error in this file - SD=7, EM=20.5, A=0
error- p=1.7298005233333384 , change p =-0.1521953099999894, t =14.536238387200001, change t = 14.5362383872
error- p=1.4777786483333384 , trough p =-5.489523437999999, t =14.778293780999999, trough t = 14.778293781
Error in this file - SD=7, EM=22.5, A=0
error- p=0.8570214850000037 , change p =-0.0416640599999954, t =14.3585941941, change t = 14.3585941941
error- p=0.4789378910000037 , trough p =-5.319210938000005, t =14.5914648846, trough t = 14.5914648846
